In [1]:
import requests
import pandas as pd

murl = "https://api.gateway.attomdata.com/"
call = "propertyapi/v1.0.0/property/address"
url = murl + call

params = {
    "postalcode": "82009",
    "page": "1",
    "pagesize" : "500"
}
headers = {
    "accept": "application/json",
    "apikey": "ea1398d715365c7396c7d2a201a6a2dc"
}

try:
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()

        
        properties = data.get("property", [])
        if not isinstance(properties, list):
            properties = [properties]

        
        df = pd.json_normalize(
            properties,
            sep='_'
        )

    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"An error occurred: {e}")


In [4]:
df = df[["address_line1", "address_line2"]]
df['address_line2'] = df['address_line2'].str.replace(r'\s\d{5}$', '', regex=True)

In [5]:
df.head()

,address_line1,address_line2
0,4009 CARLA DR,"CHEYENNE, WY"
1,4707 HICKORY PL,"CHEYENNE, WY"
2,252 E IDAHO ST,"CHEYENNE, WY"
3,9420 MICHIGAN ST,"CHEYENNE, WY"
4,4717 MORAN AVE,"CHEYENNE, WY"


In [6]:
import time


df_addresses = df



murl = "https://api.gateway.attomdata.com/"
call = "propertyapi/v1.0.0/sale/detail"
url = murl + call

headers = {
    "accept": "application/json",
    "apikey": "ea1398d715365c7396c7d2a201a6a2dc"
}


all_properties = []


for idx, row in df_addresses.iterrows():
    params = {
        "address1": row['address_line1'],
        "address2": row['address_line2']
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            properties = data.get("property", [])
            if not isinstance(properties, list):
                properties = [properties]

            
            flat = pd.json_normalize(properties, sep='_')
            all_properties.append(flat)

        else:
            print(f"[{idx}] Failed ({response.status_code}): {params}")
            continue

    except Exception as e:
        print(f"[{idx}] Error: {e}")
        continue

    
    time.sleep(0.5)


if all_properties:
    final_df = pd.concat(all_properties, ignore_index=True)
else:
    print("No data was collected.")

final_df

[0] Failed (401): {'address1': '4009 CARLA DR', 'address2': 'CHEYENNE, WY'}
[1] Failed (401): {'address1': '4707 HICKORY PL', 'address2': 'CHEYENNE, WY'}
[2] Failed (401): {'address1': '252 E IDAHO ST', 'address2': 'CHEYENNE, WY'}
[3] Failed (401): {'address1': '9420 MICHIGAN ST', 'address2': 'CHEYENNE, WY'}
[4] Failed (401): {'address1': '4717 MORAN AVE', 'address2': 'CHEYENNE, WY'}
[5] Failed (401): {'address1': '4701 ONTARIO AVE', 'address2': 'CHEYENNE, WY'}
[6] Failed (401): {'address1': '6901 WILLSHIRE BLVD', 'address2': 'CHEYENNE, WY'}
[7] Failed (401): {'address1': '4755 MORAN AVE', 'address2': 'CHEYENNE, WY'}
[8] Failed (401): {'address1': '729 APACHE ST', 'address2': 'CHEYENNE, WY'}


KeyboardInterrupt: 

In [8]:
data = pd.concat(all_properties, ignore_index=True)

ValueError: No objects to concatenate